### Elementy sztucznej inteligencji
# Projekt: Binarne Drzewa Decyzyjne
autorzy:
1. Arkadiusz Florek
2. Maciej Komosa
3. Albert Pieniądz
4. Jakub Zięba

In [2]:
import numpy as np, pandas as pd, math as mat

Funkcje do obliczania entropii dla danej konkluzji

In [3]:
def prawdopodobienstwo(wartosci):

    unikatowe_wartosci = set(wartosci)

    # tabela zawierająca częstości wystąpienia poszczególnych wartości przesłanek
    p_tab = np.zeros(len(unikatowe_wartosci))
    
    for indeks, unikatowa_wartosc in enumerate(unikatowe_wartosci):

        for wartosc in wartosci:

            if wartosc == unikatowa_wartosc:
                p_tab[indeks] += 1

    p_tab = p_tab / len(wartosci)
    return p_tab


In [4]:
def entropia(p_tab):
    I = 0
    for p in p_tab:
        if p == 0:
            continue
        else:
            I += -p*mat.log2(p)
    return I

Klasa węzła

In [5]:
class wezel:

    def __init__(self):
        self.value = None
        self.next = None
        self.childs = None

Importowanie danych

In [6]:
data = pd.read_csv("sport.csv")
data.head()
X = data.iloc[:, 0:-1].values.transpose()
Y = data.iloc[:, -1].values.transpose()

In [7]:
p_tab = []; p_buf = []
e_tab = []; e_buf = []

for przeslanka in range(len(X)):

    for konkluzja in range(len(set(Y))):

        wartosci = X[przeslanka]
        
        konkluzja += 1

        wartosci = wartosci[Y == konkluzja]
    
        p_buf.append(prawdopodobienstwo(wartosci))
        e_buf.append(entropia(p_buf[-1]))

    p_tab.append(p_buf)
    e_tab.append(e_buf)
    p_buf = []
    e_buf = []

In [8]:
# entropia inicjalna
entropia(prawdopodobienstwo(Y))

2.67943596719348

In [9]:
p_tab

[[array([0.34042553, 0.65957447]),
  array([0.24, 0.76]),
  array([0.37037037, 0.62962963]),
  array([0.81818182, 0.18181818]),
  array([0.8, 0.2]),
  array([0.69565217, 0.30434783]),
  array([0.63636364, 0.36363636])],
 [array([0.42553191, 0.23404255, 0.34042553]),
  array([0.4 , 0.32, 0.28]),
  array([0.25925926, 0.44444444, 0.2962963 ]),
  array([0.21212121, 0.42424242, 0.36363636]),
  array([0.44, 0.24, 0.32]),
  array([0.30434783, 0.30434783, 0.39130435]),
  array([0.54545455, 0.45454545])],
 [array([0.4893617 , 0.19148936, 0.31914894]),
  array([0.2 , 0.46, 0.34]),
  array([0.33333333, 0.40740741, 0.25925926]),
  array([0.36363636, 0.3030303 , 0.33333333]),
  array([0.4 , 0.48, 0.12]),
  array([0.56521739, 0.2173913 , 0.2173913 ]),
  array([0.09090909, 0.36363636, 0.54545455])],
 [array([0.27659574, 0.27659574, 0.44680851]),
  array([0.28, 0.42, 0.3 ]),
  array([0.59259259, 0.18518519, 0.22222222]),
  array([0.45454545, 0.36363636, 0.18181818]),
  array([0.24, 0.36, 0.4 ]),
  arr

In [13]:
I_tab = []
I_buf = 0

for przeslanka in p_tab:
    for indeks in range (len(przeslanka)):
        for e_indeks in range(len(e_tab)):
            I_buf += e_tab[e_indeks] * przeslanka[indeks]
        print(I_buf)
        I_buf = 0
I_tab

ValueError: operands could not be broadcast together with shapes (7,) (2,) 

In [11]:
p_tab

[[array([0.34042553, 0.65957447]),
  array([0.24, 0.76]),
  array([0.37037037, 0.62962963]),
  array([0.81818182, 0.18181818]),
  array([0.8, 0.2]),
  array([0.69565217, 0.30434783]),
  array([0.63636364, 0.36363636])],
 [array([0.42553191, 0.23404255, 0.34042553]),
  array([0.4 , 0.32, 0.28]),
  array([0.25925926, 0.44444444, 0.2962963 ]),
  array([0.21212121, 0.42424242, 0.36363636]),
  array([0.44, 0.24, 0.32]),
  array([0.30434783, 0.30434783, 0.39130435]),
  array([0.54545455, 0.45454545])],
 [array([0.4893617 , 0.19148936, 0.31914894]),
  array([0.2 , 0.46, 0.34]),
  array([0.33333333, 0.40740741, 0.25925926]),
  array([0.36363636, 0.3030303 , 0.33333333]),
  array([0.4 , 0.48, 0.12]),
  array([0.56521739, 0.2173913 , 0.2173913 ]),
  array([0.09090909, 0.36363636, 0.54545455])],
 [array([0.27659574, 0.27659574, 0.44680851]),
  array([0.28, 0.42, 0.3 ]),
  array([0.59259259, 0.18518519, 0.22222222]),
  array([0.45454545, 0.36363636, 0.18181818]),
  array([0.24, 0.36, 0.4 ]),
  arr